# BootComp Experimental Developments

This notebook is used for development of BootComp procedures

In [1]:
import Bootstrap as bs
import BootIO as io
import BootChartExtensions as ch
import MCC as mcc
import ConvFuncs as cf

In [2]:
#DEV
import Bootstrap_crn as crn

In [3]:
import pandas as pd

## 1. Parameters and setup

In [4]:
N_BOOTS = 2000
INPUT_DATA = "data/real_scenarios.csv"

In [5]:
scenario_data = crn.load_scenarios(INPUT_DATA)
N_SCENARIOS = scenario_data.shape[1]
print("Loaded data. {0} scenarios".format(N_SCENARIOS))

Loaded data. 59 scenarios


In [6]:
scenario_data.shape

(20, 59)

## 2. Analysis Procedures

In [8]:
boot_data = crn.resample_all_scenarios(scenario_data, N_BOOTS)

In [9]:
boot_data.shape

(2000, 59)

In [17]:
df_boots = pd.DataFrame(boot_data, columns = [i+1 for i in range(N_SCENARIOS)])


In [20]:
df_boots.index = df_boots.index+1

In [21]:
df_boots

,1,2,3,4,5,6,7,8,9,10,...,50,51,52,53,54,55,56,57,58,59
1,0.844170,3.083676,2.069410,1.524280,1.076681,0.962140,0.947051,0.845542,0.844170,0.844170,...,0.301646,0.259259,0.072702,0.068587,0.068586,1.355556,0.353910,0.074074,0.069959,0.069959
2,1.089849,3.539369,2.322497,1.772565,1.310700,1.245542,1.170096,1.090535,1.089849,1.089849,...,0.451989,0.282579,0.112483,0.112483,0.110424,5.996434,0.508231,0.146776,0.112483,0.111797
3,1.091084,3.311728,2.624417,1.501166,1.195336,1.134294,1.138409,1.088340,1.091084,1.091084,...,0.322908,0.165981,0.083676,0.079561,0.078875,4.312346,0.323045,0.109054,0.078875,0.078875
4,1.056928,3.349931,2.338066,1.855281,1.296297,1.213992,1.127572,1.057613,1.056928,1.056928,...,0.501097,0.236626,0.130316,0.130316,0.128943,8.088066,0.465021,0.159122,0.131687,0.130316
5,0.952949,2.996845,2.350960,1.533882,1.127161,1.035940,1.030453,0.952263,0.952949,0.952949,...,0.319616,0.202332,0.108368,0.102881,0.102195,6.177092,0.294925,0.128944,0.102195,0.102195
6,1.265432,3.635322,2.643416,1.882373,1.500000,1.456791,1.335391,1.265432,1.265432,1.265432,...,0.554733,0.291495,0.148834,0.148834,0.147461,6.494444,0.541152,0.170782,0.150206,0.148834
7,0.763375,3.110425,1.598697,1.252949,0.936214,0.868313,0.836077,0.764061,0.763375,0.763375,...,0.326063,0.126886,0.064472,0.064472,0.062413,5.838478,0.299726,0.107682,0.063100,0.062414
8,1.114266,3.667627,2.484225,1.585871,1.282305,1.235666,1.195199,1.113581,1.114266,1.114266,...,0.453086,0.246228,0.108368,0.106996,0.104938,5.551578,0.393004,0.151578,0.105624,0.104938
9,1.152126,3.410837,2.799726,1.815707,1.372291,1.289986,1.239918,1.151440,1.152126,1.152126,...,0.397119,0.269547,0.115226,0.111111,0.110424,2.672977,0.414266,0.148148,0.111797,0.111797
10,1.396845,3.871605,2.956790,2.225172,1.683540,1.578601,1.475720,1.396845,1.396845,1.396845,...,0.604253,0.281893,0.159808,0.158436,0.156378,9.102126,0.613169,0.200274,0.159122,0.157750
